DistilBERT Training code and GPU configs

In [ ]:
# Install PyTorch with CUDA 12.4 support (replace 'cu124' with the correct tag if different)
!pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu121

Defaulting to user installation because normal site-packages is not writeable
Looking in indexes: https://download.pytorch.org/whl/cu121
  Using cached https://download.pytorch.org/whl/cu121/torch-2.2.2%2Bcu121-cp312-cp312-win_amd64.whl (2454.8 MB)
  Attempting uninstall: torch
    Found existing installation: torch 2.2.2
    Uninstalling torch-2.2.2:
      Successfully uninstalled torch-2.2.2


In [ ]:
!pip install datasets
!pip install transformers

Defaulting to user installation because normal site-packages is not writeable
  Using cached fsspec-2024.2.0-py3-none-any.whl.metadata (6.8 kB)
Using cached fsspec-2024.2.0-py3-none-any.whl (170 kB)
  Attempting uninstall: fsspec
    Found existing installation: fsspec 2024.3.1
    Uninstalling fsspec-2024.3.1:
      Successfully uninstalled fsspec-2024.3.1
Defaulting to user installation because normal site-packages is not writeable


In [ ]:

from datasets import load_dataset, Dataset, concatenate_datasets
import torch
from transformers import DistilBertTokenizer, DistilBertForMaskedLM, Trainer, TrainingArguments, logging

In [ ]:
print("PyTorch version:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("CUDA version:", torch.version.cuda)
    print("Device name:", torch.cuda.get_device_name(0))

PyTorch version: 2.2.2+cu121
CUDA available: True
CUDA version: 12.1
Device name: NVIDIA GeForce RTX 3060


In [ ]:
from datasets import load_dataset, Dataset, concatenate_datasets

def load_data(file_path):
    with open(file_path, encoding="utf-8") as file:
        texts = [line.strip() for line in file if line.strip()]
    return Dataset.from_dict({"text": texts})

# Define paths to data files
data_files = {
    "open_subtitles": r"train_10m\open_subtitles.train",
    "bnc_spoken": r"train_10m\bnc_spoken.train",
    "gutenberg": r"train_10m\gutenberg.train",
    "childes": r"train_10m\childes.train",
    "simple_wiki": r"train_10m\simple_wiki.train",
    "switchboard": r"train_10m\switchboard.train"
}

# Load datasets and concatenate them
datasets = {name: load_data(path) for name, path in data_files.items()}
combined_dataset = concatenate_datasets(list(datasets.values()))

# Optionally, shuffle the dataset
combined_dataset = combined_dataset.shuffle(seed=42)


In [ ]:
from transformers import DataCollatorForLanguageModeling

tokenizer = DistilBertTokenizer.from_pretrained('distilbert-base-uncased')
data_collator = DataCollatorForLanguageModeling(
    tokenizer=tokenizer,
    mlm=True,        # Enable Masked Language Modeling
    mlm_probability=0.15
)

def tokenize_and_mask(examples):
    # Tokenize the texts
    tokenized_inputs = tokenizer(examples["text"], truncation=True, padding="max_length", max_length=512, return_tensors="np")
    # Manually adjust the length of each example if needed, or ensure your dataset doesn't exceed the max_length
    return tokenized_inputs

tokenized_datasets = combined_dataset.map(tokenize_and_mask, batched=True, remove_columns=["text"])
# Apply data collator in trainer, not in the map function



loading file vocab.txt from cache at C:\Users\rebec\.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\vocab.txt
loading file added_tokens.json from cache at None
loading file special_tokens_map.json from cache at None
loading file tokenizer_config.json from cache at C:\Users\rebec\.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\tokenizer_config.json
loading file tokenizer.json from cache at C:\Users\rebec\.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\tokenizer.json
loading configuration file config.json from cache at C:\Users\rebec\.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_d

Map:   0%|          | 0/1165314 [00:00<?, ? examples/s]

In [ ]:
!pip install transformers[torch] accelerate -U --force-reinstall

Defaulting to user installation because normal site-packages is not writeable
  Using cached accelerate-0.29.2-py3-none-any.whl.metadata (18 kB)
  Using cached transformers-4.39.3-py3-none-any.whl.metadata (134 kB)
  Using cached filelock-3.13.4-py3-none-any.whl.metadata (2.8 kB)
  Using cached huggingface_hub-0.22.2-py3-none-any.whl.metadata (12 kB)
  Using cached numpy-1.26.4-cp312-cp312-win_amd64.whl.metadata (61 kB)
  Using cached packaging-24.0-py3-none-any.whl.metadata (3.2 kB)
  Using cached PyYAML-6.0.1-cp312-cp312-win_amd64.whl.metadata (2.1 kB)
  Using cached regex-2024.4.16-cp312-cp312-win_amd64.whl.metadata (41 kB)
  Using cached requests-2.31.0-py3-none-any.whl.metadata (4.6 kB)
  Using cached tokenizers-0.15.2-cp312-none-win_amd64.whl.metadata (6.8 kB)
  Using cached safetensors-0.4.3-cp312-none-win_amd64.whl.metadata (3.9 kB)
  Using cached tqdm-4.66.2-py3-none-any.whl.metadata (57 kB)
  Using cached torch-2.2.2-cp312-cp312-win_amd64.whl.metadata (26 kB)
  Using cached p

  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
  You can safely remove it manually.
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datasets 2.18.0 requires fsspec[http]<=2024.2.0,>=2023.1.0, but you have fsspec 2024.3.1 which is incompatible.
torchaudio 2.2.2+cu121 requires torch==2.2.2+cu121, but you have torch 2.2.2 which is incompatible.
torchvision 0.17.2+cu121 requires torch==2.2.2+cu121, but you have torch 2.2.2 which is incompatible.


In [ ]:
!pip show accelerate transformers


Name: accelerate
Version: 0.29.2
Summary: Accelerate
Home-page: https://github.com/huggingface/accelerate
Author: The HuggingFace team
Author-email: zach.mueller@huggingface.co
License: Apache
Location: C:\Users\rebec\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages
Requires: huggingface-hub, numpy, packaging, psutil, pyyaml, safetensors, torch
Required-by: 
---
Name: transformers
Version: 4.39.3
Summary: State-of-the-art Machine Learning for JAX, PyTorch and TensorFlow
Home-page: https://github.com/huggingface/transformers
Author: The Hugging Face team (past and future) with the help of all our contributors (https://github.com/huggingface/transformers/graphs/contributors)
Author-email: transformers@huggingface.co
License: Apache 2.0 License
Location: C:\Users\rebec\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages
Requires: filelock, hugg

In [ ]:
print(torch.__version__)

2.2.2+cu121


In [ ]:
import sys
print(sys.executable)


C:\Users\rebec\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\python.exe


In [ ]:
from accelerate import Accelerator

In [ ]:
model = DistilBertForMaskedLM.from_pretrained('distilbert-base-uncased')

# Set the logger to use the INFO level
logging.set_verbosity_info()

training_args = TrainingArguments(
    output_dir="./results",
    num_train_epochs=3,
    logging_dir='./logs',  # Directory for storing logs
    logging_steps=100,  # Log training info every 100 steps
    per_device_train_batch_size=8,  # Adjust based on your GPU memory
    save_steps=10_000,
    save_total_limit=2,
    report_to="all",  # Ensures logging to both TensorBoard and stdout
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets,
    data_collator=data_collator  # Pass the data collator here to handle dynamic masking
)


loading configuration file config.json from cache at C:\Users\rebec\.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\config.json
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.39.3",
  "vocab_size": 30522
}

loading weights file model.safetensors from cache at C:\Users\rebec\.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\6cdc0aad91f5ae2e6712e91bc7b65d1cf5c05411\model.safetensors
All model checkpoint weights were used when initializing DistilBertForMaskedLM.

All the weights of DistilBertFor

In [ ]:
trainer.train()

***** Running training *****
  Num examples = 1,165,314
  Num Epochs = 3
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 436,995
  Number of trainable parameters = 66,985,530


  0%|          | 0/436995 [00:00<?, ?it/s]

{'loss': 2.7525, 'grad_norm': 33.065460205078125, 'learning_rate': 4.998855822148995e-05, 'epoch': 0.0}
{'loss': 2.6615, 'grad_norm': 26.558879852294922, 'learning_rate': 4.99771164429799e-05, 'epoch': 0.0}
{'loss': 2.5155, 'grad_norm': 30.26500129699707, 'learning_rate': 4.9965674664469846e-05, 'epoch': 0.0}
{'loss': 2.5828, 'grad_norm': 23.92010498046875, 'learning_rate': 4.9954232885959794e-05, 'epoch': 0.0}
{'loss': 2.5406, 'grad_norm': 23.818222045898438, 'learning_rate': 4.994279110744974e-05, 'epoch': 0.0}
{'loss': 2.603, 'grad_norm': 25.47873306274414, 'learning_rate': 4.993134932893969e-05, 'epoch': 0.0}
{'loss': 2.2571, 'grad_norm': 24.167726516723633, 'learning_rate': 4.991990755042964e-05, 'epoch': 0.0}
{'loss': 2.3206, 'grad_norm': 17.598976135253906, 'learning_rate': 4.9908465771919586e-05, 'epoch': 0.01}
{'loss': 2.6281, 'grad_norm': 32.55040740966797, 'learning_rate': 4.9897023993409534e-05, 'epoch': 0.01}
{'loss': 2.5626, 'grad_norm': 21.74773597717285, 'learning_rate'

Saving model checkpoint to ./results\checkpoint-10000
Configuration saved in ./results\checkpoint-10000\config.json


{'loss': 2.3511, 'grad_norm': 18.42654037475586, 'learning_rate': 4.885582214899484e-05, 'epoch': 0.07}


...

Model weights saved in ./results\checkpoint-430000\model.safetensors
Deleting older checkpoint [results\checkpoint-410000] due to args.save_total_limit


{'loss': 2.0743, 'grad_norm': 14.663187026977539, 'learning_rate': 7.889106282680581e-07, 'epoch': 2.95}
{'loss': 2.173, 'grad_norm': 29.198667526245117, 'learning_rate': 7.774688497580064e-07, 'epoch': 2.95}
{'loss': 2.2421, 'grad_norm': 14.411456108093262, 'learning_rate': 7.660270712479548e-07, 'epoch': 2.95}
{'loss': 2.0204, 'grad_norm': 12.599611282348633, 'learning_rate': 7.545852927379032e-07, 'epoch': 2.95}
{'loss': 2.1024, 'grad_norm': 16.336156845092773, 'learning_rate': 7.431435142278516e-07, 'epoch': 2.96}
{'loss': 2.2775, 'grad_norm': 18.42116355895996, 'learning_rate': 7.317017357178e-07, 'epoch': 2.96}
{'loss': 2.0329, 'grad_norm': 25.307931900024414, 'learning_rate': 7.202599572077484e-07, 'epoch': 2.96}
{'loss': 2.0906, 'grad_norm': 26.186702728271484, 'learning_rate': 7.088181786976968e-07, 'epoch': 2.96}
{'loss': 2.0312, 'grad_norm': 26.278783798217773, 'learning_rate': 6.973764001876451e-07, 'epoch': 2.96}
{'loss': 1.9955, 'grad_norm': 33.63789367675781, 'learning_r



Training completed. Do not forget to share your model on huggingface.co/models =)




{'train_runtime': 151672.5055, 'train_samples_per_second': 23.049, 'train_steps_per_second': 2.881, 'train_loss': 2.235039036472468, 'epoch': 3.0}


TrainOutput(global_step=436995, training_loss=2.235039036472468, metrics={'train_runtime': 151672.5055, 'train_samples_per_second': 23.049, 'train_steps_per_second': 2.881, 'train_loss': 2.235039036472468, 'epoch': 3.0})

In [ ]:
model.save_pretrained("final model2")


Configuration saved in final model2\config.json


Model weights saved in final model2\model.safetensors


In [2]:
model.save_pretrained("final model 4/28", safe_serialization=False)

NameError: name 'model' is not defined